In [135]:
import csv
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn import preprocessing
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

In [136]:
data_path = os.path.abspath('.') + '/../data/'
train = pd.read_csv(data_path + "train.csv")
test = pd.read_csv(data_path + "test.csv")
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

In [137]:
test.head()
id_list = test['ID_LAT_LON_YEAR_WEEK']

In [138]:
train.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


In [139]:
train.shape, test.shape, sample_submission.shape

((79023, 76), (24353, 75), (24353, 2))

In [140]:
train = train.loc[:,["latitude", "longitude", "year", "week_no", "emission"]]
test = test.loc[:,["latitude", "longitude", "year", "week_no"]]

$latitude$和$longitude$是重要特征，之后补特征分析。

K-Means聚类

In [141]:
k_model = KMeans(n_clusters=6)
train_x = train.groupby(by=['latitude', 'longitude'], as_index=False)['emission'].mean()
k_model.fit(train_x)
k_Means = k_model.predict(train_x)
train_x['k_Means'] = k_Means

train = train.merge(train_x[['latitude', 'longitude', 'k_Means']], on=['latitude', 'longitude'])
test = test.merge(train_x[['latitude', 'longitude', 'k_Means']], on=['latitude', 'longitude'])

train.head()

C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,latitude,longitude,year,week_no,emission,k_Means
0,-0.51,29.29,2019,0,3.750994,1
1,-0.51,29.29,2019,1,4.025176,1
2,-0.51,29.29,2019,2,4.231381,1
3,-0.51,29.29,2019,3,4.305286,1
4,-0.51,29.29,2019,4,4.347317,1


Predict 2022

In [142]:
cat_params = {
    
    'n_estimators': 799, 
    'learning_rate': 0.09180872710592884,
    'depth': 8, 
    'l2_leaf_reg': 1.0242996861886846, 
    'subsample': 0.38227256755249117, 
    'colsample_bylevel': 0.7183481537623551,
    'random_state': 42,
    "silent": True,
}

clusters = train["k_Means"].unique()

for i in clusters:
    train_c = train[train["k_Means"] == i]
    test_c = test[test["k_Means"] == i].drop(columns = ["k_Means"])
    X = train_c.drop(columns = ["emission", "k_Means"])
    y = train_c["emission"].copy()

    catboost_reg = CatBoostRegressor(**cat_params)
    catboost_reg.fit(X, y)
    catboost_pred = catboost_reg.predict(test_c)

    test.loc[test["k_Means"] == i, "emission"] = catboost_pred
    

In [ ]:
test['ID_LAT_LON_YEAR_WEEK'] = id_list
test = test.loc[:,["ID_LAT_LON_YEAR_WEEK", "emission"]]
test.to_csv(data_path + "submission.csv", index=False)